In [ ]:
import pandas as pd

PATH = 'data/'


In [ ]:
train_essays = pd.read_csv(PATH + 'llm-detect-ai-generated-text/train_essays.csv')
train_essays

In [ ]:
Radek_data_gpt_3_5 = pd.read_csv(PATH + 'llm-generated-essays/ai_generated_train_essays.csv')
Radek_data_gpt_4   = pd.read_csv(PATH + 'llm-generated-essays/ai_generated_train_essays_gpt-4.csv')

In [ ]:
PERSUADE_corpus = pd.read_csv(PATH + "persaude-corpus-2/persuade_2.0_human_scores_demo_id_github.csv")
PERSUADE_corpus = PERSUADE_corpus["full_text"].to_frame()
PERSUADE_corpus["generated"] = 0 # human data
PERSUADE_corpus = PERSUADE_corpus.rename(columns = {'full_text':'text'})
PERSUADE_corpus.shape

In [ ]:
llama_70b_dataset = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_70b_v1.csv")
llama_70b_dataset = llama_70b_dataset["generated_text"].to_frame()
llama_70b_dataset["generated"] = 1 # AI-LLM data
llama_70b_dataset = llama_70b_dataset.rename(columns = {'generated_text':'text'})

falcon_180b_dataset = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/falcon_180b_v1.csv")
falcon_180b_dataset = falcon_180b_dataset["generated_text"].to_frame()
falcon_180b_dataset["generated"] = 1 # AI-LLM data
falcon_180b_dataset = falcon_180b_dataset.rename(columns = {'generated_text':'text'})


In [ ]:
daigt_external_dataset = pd.read_csv(PATH + "daigt-external-dataset/daigt_external_dataset.csv")
daigt_external_dataset = daigt_external_dataset['text'].to_frame()
daigt_external_dataset["generated"] = 1
daigt_external_dataset.shape

In [ ]:
import pandas as pd
import re
import random
import numpy as np
from tqdm import tqdm 


BRACKET_SYMBOL = ['[', ']', '(', ')', '{', '}']
SPECIAL_CHARACTERS = ['.', '+', '*', '?', '^', '$', '(', ')', '[', ']', '{', '}', '|', '\\']
CHARACTERS = 'abcdefghijklmnopqrstuvwxyz'
VOWEL = 'ueoai'
CONSONANTS = 'bcdfghjklmnpqrstvwxz'



def normalize_text(text):
    text = text.replace('-', '')
    for symbol in BRACKET_SYMBOL:
        text = text.replace(symbol, f' {symbol} ')

    text = re.sub(' +', ' ', text)
    return text

def is_word(word):
    for c in SPECIAL_CHARACTERS:
        if c in word:
            return False
    return True

class Noise:    
    def remove_consonant(self, words, rate=0.2):
        for i, word in enumerate(words):
            if is_word(word):
                c = random.choice(CONSONANTS)
                prob = np.random.uniform(0, 1, 1)
                if prob[0] < rate:
                    words[i] = words[i].replace(c, '')

        return words
    
    def replace_consonant(self, words):
        v = random.choice(VOWEL)
        c = random.choice(CONSONANTS)
        for i, word in enumerate(words):
            if is_word(word):
                words[i] = words[i].replace(c, v)
                words[i] = words[i].replace(c.upper(), v.upper())
        return words
    
    def remove_space(self, words, n=3):
        sentence = ' '.join(words)
        space_ids = np.random.randint(0, len(words)-2, n)
        phrases = []
        for i in space_ids:
            phrases.append(f'{words[i]} {words[i+1]}')
        for phrase in phrases:
            sentence = sentence.replace(phrase, phrase.replace(' ', ''))
        return sentence.split()
    
    def insert_vowel(self, words, rate=0.4):
        v = random.choice(VOWEL)
        unique_words = list(set(words))
        chosen_words = np.random.choice(unique_words, int(rate*len(unique_words)))
        inserted_words = []
        for word in chosen_words:
            id = random.randint(0, len(word)+1)
            inserted_words.append(word[:id] + v + word[id:])
        sentence = ' '.join(words)
        for w, r in zip(chosen_words, inserted_words):
            sentence = sentence.replace(w, r)
        words = sentence.split()
#         print(' '.join(words))
#         print('inserted_words: ', inserted_words)
        return words
    
    def randomly_lower(self, words, rate=0.3):
        special_words = []
        for i, word in enumerate(words):
            if i == 0:
                continue
            if word[0] == word[0].upper() and not is_word(words[i-1]):
                special_words.append(word)
        n = int(rate*len(special_words)+1)
        if n > len(special_words):
            return words
        chosen_words = np.random.choice(special_words, n)
        
        sentence = ' '.join(words)
        for word in chosen_words:
            sentence = sentence.replace(word, word.lower())
        
        return sentence.split()
    
    def remove_apostrophe(self, words, rate=0.2):
        sentence = ' '.join(words)
        apost_phrase = re.findall('[^ ]*\'[^ ]*', sentence)
        if int(rate*len(apost_phrase)+1) > len(apost_phrase):
            return words
        chosen_words = np.random.choice(apost_phrase, int(rate*len(apost_phrase)+1))
        for word in chosen_words:
            sentence = sentence.replace(word, word.replace("'", ""))
        return sentence.split()
    
    def add_noise_to_corpus(self, corpus, rate=0.9):
        prob = np.random.uniform(0, 1, len(corpus))
        choice = prob > rate
        new_corpus = []
        for i in tqdm(range(len(corpus))):
            if choice[i]:
                new_corpus.append(corpus[i])
            else:
                words = corpus[i].split()
                noise_id = np.random.randint(0, 2, 6)
                if noise_id[0] == 1:
                    words = self.replace_consonant(words)
                if noise_id[1] == 1:
                    words = self.remove_space(words)
                if noise_id[2] == 1:
                    words = self.insert_vowel(words)
                if noise_id[3] == 1:
                    words = self.randomly_lower(words)
                if noise_id[4] == 1:
                    words = self.remove_apostrophe(words)
                if noise_id[5] == 1:
                    words = self.remove_consonant(words)
                new_corpus.append(' '.join(words))
        return new_corpus

In [ ]:
dataset = pd.concat([train_essays,
                      Radek_data_gpt_3_5, 
                     Radek_data_gpt_4, 
                     PERSUADE_corpus, 
                     llama_70b_dataset, 
                     falcon_180b_dataset,
                     daigt_external_dataset
                    ], 
                    ignore_index=True)

dataset["generated"].value_counts()

In [ ]:
noise = Noise()
corpus = noise.add_noise_to_corpus(dataset["text"].tolist())
dataset["text"] = corpus
# dataset["text"] = [normalize_text(c) for c in corpus]

In [ ]:
# very rudimentary cleaning
def cleaning(dataset):
    
    dataset['text'] = dataset['text'].str.strip()
    dataset["text"] = dataset["text"].replace('\\n',' ')
    dataset["text"] = dataset["text"].str.split('ubject: ').str[-1].str.strip()
    dataset["text"] = dataset["text"].str.split('Zip').str[-1].str.strip()
    dataset["text"] = dataset["text"].str.split('ZIP').str[-1].str.strip()
#     dataset = dataset.rename(columns = {'generated':'label'})
#     dataset = dataset.drop(["id","prompt_id"], axis=1)
    
    return dataset

In [ ]:
dataset = cleaning(dataset)
dataset.to_csv('train_essays_3.0.csv', index=False)